In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
## 设置字体
from matplotlib.font_manager import FontProperties
import pandas as pd
fonts = FontProperties(fname = "/Library/Fonts/华文细黑.ttf",size=14)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [144]:
train = pd.read_csv("../data/train.csv")
train = train.dropna()
train.head()

,content,label
0,什么是电气一次设备和一次回路,1
1,一次设备是指直接生产输送和分配电能的高压电气设备它包括发电机变压器断路器隔离开关自动开关接触...,0
2,由一次设备相互连接构成发电输电配电或进行其它生产的电气回路称为一次回路或一次接线系统,0
3,什么是电气二次设备和二次回路,1
4,二次设备是指对一次设备的工作进行监测控制调节保护以及为运行维护人员提供运行工况或生产指挥信号...,0


In [145]:
train.shape,train[train['label'] == 1].shape

((5912, 2), (1593, 2))

In [146]:
def filter_abnormal(data):
    if len(data.content) > 30 or data.label==1:
        return data
new_train = train.apply(lambda x:filter_abnormal(x),axis=1).dropna()
new_train.shape

(3539, 2)

In [147]:
new_train.shape,new_train[train['label'] == 1].shape

D:\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


((3539, 2), (1593, 2))

In [12]:
new_train = train.copy()

In [148]:
from LAC import LAC
lac = LAC(mode='lac')
with open("../data/stopwords.txt",'rb') as f:
    stop_words = f.read().decode("utf-8")
stop_words = stop_words.split("\r\n")
def cut_words(sentence):
    words_list = []
    words,typ = lac.run(sentence)
    for i in range(len(words)):
        if typ[i] != "n" and typ[i] != 'm':
#         if typ[i] != "n" and typ[i] != 'm' and words[i] not in stop_words:
            words_list.append(words[i])
    return " ".join(words_list)
new_train['tokens'] = new_train.content.apply(lambda x: cut_words(x))

In [92]:
new_train

,content,label,tokens
0,什么是电气一次设备和一次回路,1.0,什么 是 和
1,一次设备是指直接生产输送和分配电能的高压电气设备它包括发电机变压器断路器隔离开关自动开关接触...,0.0,是 指 直接 生产 输送 和 分配 的 它 包括 发电机变压器断路器 隔离开关自动开关接触器...
2,由一次设备相互连接构成发电输电配电或进行其它生产的电气回路称为一次回路或一次接线系统,0.0,由 相互 连接 构成 发电 输电 配电 或 进行 其它 生产 的 电气回路 称为 或 接线
3,什么是电气二次设备和二次回路,1.0,什么 是 和
4,二次设备是指对一次设备的工作进行监测控制调节保护以及为运行维护人员提供运行工况或生产指挥信号...,0.0,是 指 对 的 进行 监测 控制 调节 保护 以及 为 运行 维护 提供 运行 或 生产 指...
...,...,...,...
7201,凹处地方造成它所覆盖的处所与空气隔绝表现为缺氧容易污染环境又不易被,0.0,造成 它 所 覆盖 的 与 隔绝 表现 为 缺氧 容易 污染 又 不易 被
7205,水解和可水解物杂质成份增加导致介质质量变差甚至不合格泄漏也造成浪费,0.0,水解 和 可 增加 导致 变 差 甚至 不合格 泄漏 也 造成 浪费
7208,持在设计规定的范围之内避免泄漏才能使电气设备达到或维持在设计的绝缘水,0.0,持 在 设计 规定 的 之 内 避免 泄漏 才能 使 达到 或 维持 在 设计 的 绝缘
7210,防毒面具是什么？,1.0,是 什么 ？


In [149]:
train_y = new_train.label.astype("int").values
new_train.head()

,content,label,tokens
0,什么是电气一次设备和一次回路,1.0,什么 是 和
1,一次设备是指直接生产输送和分配电能的高压电气设备它包括发电机变压器断路器隔离开关自动开关接触...,0.0,是 指 直接 生产 输送 和 分配 的 它 包括 发电机变压器断路器 隔离开关自动开关接触器...
2,由一次设备相互连接构成发电输电配电或进行其它生产的电气回路称为一次回路或一次接线系统,0.0,由 相互 连接 构成 发电 输电 配电 或 进行 其它 生产 的 电气回路 称为 或 接线
3,什么是电气二次设备和二次回路,1.0,什么 是 和
4,二次设备是指对一次设备的工作进行监测控制调节保护以及为运行维护人员提供运行工况或生产指挥信号...,0.0,是 指 对 的 进行 监测 控制 调节 保护 以及 为 运行 维护 提供 运行 或 生产 指...


In [159]:
le = LabelEncoder()
train_y = le.fit_transform(new_train.label).reshape(-1,1)
ohe = OneHotEncoder()
train_y = ohe.fit_transform(train_y).toarray()
train_y

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

In [160]:
## 使用Tokenizer对词组进行编码
## 当我们创建了一个Tokenizer对象后，使用该对象的fit_on_texts()函数，以空格去识别每个词,
## 可以将输入的文本中的每个词编号，编号是根据词频的，词频越大，编号越小。
max_words = 10000
max_len = 10
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(new_train.tokens)

## 使用word_index属性可以看到每次词对应的编码
## 使用word_counts属性可以看到每个词对应的频数
for ii,iterm in enumerate(tok.word_index.items()):
    if ii < 10:
        print(iterm)
    else:
        break
print("= = = = = =")
for ii,iterm in enumerate(tok.word_counts.items()):
    if ii < 10:
        print(iterm)
    else:
        break

('的', 1)
('在', 2)
('是', 3)
('？', 4)
('和', 5)
('有', 6)
('中', 7)
('应', 8)
('交换', 9)
('或', 10)
= = = = = =
('什么', 466)
('是', 1351)
('和', 1103)
('指', 70)
('直接', 53)
('生产', 101)
('输送', 5)
('分配', 10)
('的', 7027)
('它', 130)


In [161]:
## 对每个词编码之后，每句新闻中的每个词就可以用对应的编码表示，即每条新闻可以转变成一个向量了：
train_seq = tok.texts_to_sequences(new_train.tokens)
## 将每个序列调整为相同的长度
train_seq_mat = sequence.pad_sequences(train_seq,maxlen=max_len)
train_seq_mat.shape

(3539, 10)

In [162]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_seq_mat,train_y, random_state=2,train_size=0.8)

In [163]:
X_train.shape

(2831, 10)

In [164]:
X_test.shape

(708, 10)

In [165]:
model = Sequential()
model.add(Embedding(max_words+1,60,input_length=max_len))
def build_model(num_layers,hidden_size,max_words=max_words,max_len=max_len,num_classes=2):
        '''
        hidden_size：LSTM隐藏层
        return_sequences:Boolen,return_sequences = True，
        则RNN层还可以返回每个样本的整个输出序列（每个样本每个时间步一个向量）。
        该输出的形状是（batch_size，timesteps，units）
        input_shape:输入训练数据的维度
        '''
        model = Sequential([
            Embedding(max_words+1,60,input_length=max_len),
            LSTM(hidden_size, return_sequences=True),
        ])
        for i in range(2,num_layers+1):
            if i == num_layers:
                model.add(LSTM(hidden_size, return_sequences=False))
            else:
                model.add(LSTM(hidden_size, return_sequences=True))
        model.add(Dense(128,activation="relu",name="FC1"))
        model.add(Dropout(0.5))
        model.add(Dense(num_classes,activation="sigmoid",name="FC2"))
        return model

In [166]:
model = build_model(3,64)
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 10, 60)            600060    
_________________________________________________________________
lstm_12 (LSTM)               (None, 10, 64)            32000     
_________________________________________________________________
lstm_13 (LSTM)               (None, 10, 64)            33024     
_________________________________________________________________
lstm_14 (LSTM)               (None, 64)                33024     
_________________________________________________________________
FC1 (Dense)                  (None, 128)               8320      
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
FC2 (Dense)                  (None, 2)                

In [167]:
model.compile(optimizer=Adam(0.001),
        loss='binary_crossentropy',
        metrics=['accuracy'])

In [94]:
logdir = "..\logs\\" + datetime.datetime.now().strftime("%Y%m%d")

if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,
                                 "fashion_mnist_model.h5")

callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=1, profile_batch = 100000000),
#     tf.keras.callbacks.ModelCheckpoint(output_model_file,
#                                     save_best_only = True),
#     tf.keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3),
]
# history = model.fit(x_train_scaled, y_train, epochs=10,
#                     validation_data=(x_valid_scaled, y_valid),
#                     callbacks = callbacks)

In [44]:
import datetime
import tensorflow as tf
import os
log_dir = "..\logs\\" + datetime.datetime.now().strftime("%Y%m%d")
folder = os.path.exists(log_dir)
if not folder:
    os.makedirs(log_dir)
callbacks = [tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=1, profile_batch = 100000000)]

NameError: name 'logdir' is not defined

In [168]:
model_fit = model.fit(X_train,y_train,batch_size=1024,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
3/3 [==============================] - 1s 224ms/step - loss: 0.6919 - accuracy: 0.6817 - val_loss: 0.6887 - val_accuracy: 0.7288
Epoch 2/100
3/3 [==============================] - 0s 17ms/step - loss: 0.6864 - accuracy: 0.7354 - val_loss: 0.6800 - val_accuracy: 0.7994
Epoch 3/100
3/3 [==============================] - 0s 17ms/step - loss: 0.6748 - accuracy: 0.8237 - val_loss: 0.6615 - val_accuracy: 0.8588
Epoch 4/100
3/3 [==============================] - 0s 18ms/step - loss: 0.6503 - accuracy: 0.8834 - val_loss: 0.6227 - val_accuracy: 0.8842
Epoch 5/100
3/3 [==============================] - 0s 17ms/step - loss: 0.5977 - accuracy: 0.9039 - val_loss: 0.5424 - val_accuracy: 0.8941
Epoch 6/100
3/3 [==============================] - 0s 17ms/step - loss: 0.4932 - accuracy: 0.9120 - val_loss: 0.4036 - val_accuracy: 0.8983
Epoch 7/100
3/3 [==============================] - 0s 17ms/step - loss: 0.3420 - accuracy: 0.9195 - val_loss: 0.2679 - val_accuracy: 0.9110
Epoch 8/100
3/3 [==

3/3 [==============================] - 0s 17ms/step - loss: 1.7623e-04 - accuracy: 1.0000 - val_loss: 0.5527 - val_accuracy: 0.9237
Epoch 59/100
3/3 [==============================] - 0s 18ms/step - loss: 1.6622e-04 - accuracy: 1.0000 - val_loss: 0.5567 - val_accuracy: 0.9237
Epoch 60/100
3/3 [==============================] - 0s 17ms/step - loss: 1.6286e-04 - accuracy: 1.0000 - val_loss: 0.5606 - val_accuracy: 0.9237
Epoch 61/100
3/3 [==============================] - 0s 18ms/step - loss: 1.5516e-04 - accuracy: 1.0000 - val_loss: 0.5643 - val_accuracy: 0.9237
Epoch 62/100
3/3 [==============================] - 0s 17ms/step - loss: 1.5298e-04 - accuracy: 1.0000 - val_loss: 0.5678 - val_accuracy: 0.9237
Epoch 63/100
3/3 [==============================] - 0s 17ms/step - loss: 1.7569e-04 - accuracy: 1.0000 - val_loss: 0.5713 - val_accuracy: 0.9237
Epoch 64/100
3/3 [==============================] - 0s 17ms/step - loss: 1.2842e-04 - accuracy: 1.0000 - val_loss: 0.5746 - val_accuracy: 0.923

In [215]:
model_fit = model.fit(train_seq_mat,train_y,batch_size=1024,epochs=300)

Epoch 1/300
2/2 [==============================] - 0s 4ms/step - loss: 0.6924 - sparse_categorical_accuracy: 0.5075
Epoch 2/300
2/2 [==============================] - 0s 6ms/step - loss: 0.6839 - sparse_categorical_accuracy: 0.8721
Epoch 3/300
2/2 [==============================] - 0s 6ms/step - loss: 0.6654 - sparse_categorical_accuracy: 0.9579
Epoch 4/300
2/2 [==============================] - 0s 7ms/step - loss: 0.6220 - sparse_categorical_accuracy: 0.9968
Epoch 5/300
2/2 [==============================] - 0s 6ms/step - loss: 0.5282 - sparse_categorical_accuracy: 1.0000
Epoch 6/300
2/2 [==============================] - 0s 6ms/step - loss: 0.4029 - sparse_categorical_accuracy: 1.0000
Epoch 7/300
2/2 [==============================] - 0s 7ms/step - loss: 0.3068 - sparse_categorical_accuracy: 0.9992
Epoch 8/300
2/2 [==============================] - 0s 6ms/step - loss: 0.2199 - sparse_categorical_accuracy: 1.0000
Epoch 9/300
2/2 [==============================] - 0s 6ms/step - loss: 0

2/2 [==============================] - 0s 5ms/step - loss: 5.6856e-05 - sparse_categorical_accuracy: 1.0000
Epoch 137/300
2/2 [==============================] - 0s 7ms/step - loss: 5.8212e-05 - sparse_categorical_accuracy: 1.0000
Epoch 138/300
2/2 [==============================] - 0s 6ms/step - loss: 8.0688e-05 - sparse_categorical_accuracy: 1.0000
Epoch 139/300
2/2 [==============================] - 0s 6ms/step - loss: 6.8016e-05 - sparse_categorical_accuracy: 1.0000
Epoch 140/300
2/2 [==============================] - 0s 6ms/step - loss: 7.0328e-05 - sparse_categorical_accuracy: 1.0000
Epoch 141/300
2/2 [==============================] - 0s 6ms/step - loss: 6.0625e-05 - sparse_categorical_accuracy: 1.0000
Epoch 142/300
2/2 [==============================] - 0s 6ms/step - loss: 5.9052e-05 - sparse_categorical_accuracy: 1.0000
Epoch 143/300
2/2 [==============================] - 0s 6ms/step - loss: 6.3531e-05 - sparse_categorical_accuracy: 1.0000
Epoch 144/300
2/2 [===================

2/2 [==============================] - 0s 6ms/step - loss: 3.6988e-05 - sparse_categorical_accuracy: 1.0000
Epoch 271/300
2/2 [==============================] - 0s 6ms/step - loss: 1.9484e-05 - sparse_categorical_accuracy: 1.0000
Epoch 272/300
2/2 [==============================] - 0s 6ms/step - loss: 2.7099e-05 - sparse_categorical_accuracy: 1.0000
Epoch 273/300
2/2 [==============================] - 0s 6ms/step - loss: 1.3864e-05 - sparse_categorical_accuracy: 1.0000
Epoch 274/300
2/2 [==============================] - 0s 6ms/step - loss: 2.0918e-05 - sparse_categorical_accuracy: 1.0000
Epoch 275/300
2/2 [==============================] - 0s 7ms/step - loss: 2.1958e-05 - sparse_categorical_accuracy: 1.0000
Epoch 276/300
2/2 [==============================] - 0s 7ms/step - loss: 2.6259e-05 - sparse_categorical_accuracy: 1.0000
Epoch 277/300
2/2 [==============================] - 0s 6ms/step - loss: 1.8857e-05 - sparse_categorical_accuracy: 1.0000
Epoch 278/300
2/2 [===================

In [178]:
new_train

,content,label,tokens
0,什么是电气一次设备和一次回路,1.0,什么 是 和
1,一次设备是指直接生产输送和分配电能的高压电气设备它包括发电机变压器断路器隔离开关自动开关接触...,0.0,是 指 直接 生产 输送 和 分配 的 它 包括 发电机变压器断路器 隔离开关自动开关接触器...
2,由一次设备相互连接构成发电输电配电或进行其它生产的电气回路称为一次回路或一次接线系统,0.0,由 相互 连接 构成 发电 输电 配电 或 进行 其它 生产 的 电气回路 称为 或 接线
3,什么是电气二次设备和二次回路,1.0,什么 是 和
4,二次设备是指对一次设备的工作进行监测控制调节保护以及为运行维护人员提供运行工况或生产指挥信号...,0.0,是 指 对 的 进行 监测 控制 调节 保护 以及 为 运行 维护 提供 运行 或 生产 指...
...,...,...,...
7201,凹处地方造成它所覆盖的处所与空气隔绝表现为缺氧容易污染环境又不易被,0.0,造成 它 所 覆盖 的 与 隔绝 表现 为 缺氧 容易 污染 又 不易 被
7205,水解和可水解物杂质成份增加导致介质质量变差甚至不合格泄漏也造成浪费,0.0,水解 和 可 增加 导致 变 差 甚至 不合格 泄漏 也 造成 浪费
7208,持在设计规定的范围之内避免泄漏才能使电气设备达到或维持在设计的绝缘水,0.0,持 在 设计 规定 的 之 内 避免 泄漏 才能 使 达到 或 维持 在 设计 的 绝缘
7210,防毒面具是什么？,1.0,是 什么 ？


In [181]:
train_seq_mat[0]

array([ 0,  0,  0,  0,  0,  0,  0, 15,  3,  5])

In [268]:
test_data = cut_words("请问我能不能把户口迁到无锡？")

In [269]:
test_data

'请问 我 能不能 把 迁 到 无锡 ？'

In [270]:
val_seq = tok.texts_to_sequences([test_data])
## 将每个序列调整为相同的长度
val_seq_mat = sequence.pad_sequences(val_seq,maxlen=max_len)
## 对验证集进行预测
val_pre = model.predict(val_seq_mat)

In [271]:
val_pre.argsort()[0][1]

1

In [255]:
val_pre = model.predict(X_test)

In [256]:
val_pre.shape

(630, 2)

In [264]:
X_test

array([[  0,   0,   0, ..., 566,  31,   9],
       [  0,   0,   0, ..., 556,  31, 128],
       [110,   2, 431, ..., 315, 178, 123],
       ...,
       [  0,   0,   0, ..., 205, 936,  11],
       [129, 153,  91, ...,  24, 742,   2],
       [180,  23,  93, ...,  68, 404, 430]])

In [261]:
y_test

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,

In [262]:
confm = metrics.confusion_matrix(y_pred = np.argmax(val_pre,axis=1),y_true=y_test)

In [263]:
confm

array([[305,   0],
       [  0, 325]], dtype=int64)